In [348]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
import random

In [349]:
# Price of Call Option function
def get_call( T, r, d, S0,K, sigma):
    if K==0 or T==0 or sigma ==0:
        print('Not a valid input')
    elif d == 0:
        d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1-sigma*np.sqrt(T)
        call = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return call
    elif d < 0:
        print('Not a valid dividend input')
    else:
        d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1-sigma*np.sqrt(T)
        print(d1)
        call = np.exp(-d*T)*S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return(call)

In [350]:
# BS Delta of a call option
def get_delta_call(T,r,d,S0,K,sigma):
    d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    delta= norm.cdf(d1)
    return delta , d1

In [358]:
tau =[1.00,0.92,0.83,0.75,0.67,0.58,0.50,0.42,0.33,0.25,0.17,0.08,0.000001]
z=[0]
for i in range (0,12):
    z.append(np.random.randn())
S=[100]
periods = 12
r=0
K=106
T=1
sigma=0.17
C0=get_call(T=1, r=0, d=0, S0=100,K=106, sigma=0.17)
d1=[]
delta=[]
ds=[]
portfolio=[]
bs=[]
pnl=[]

#generate random stock process
S= [100 ,S[0]*np.exp((r-0.5*sigma**2)/periods + sigma*z[2]/np.sqrt(periods))]
for j in range(1,12):
    S.append(S[j]*np.exp((r-0.5*sigma**2)/periods + sigma*z[j+1]/np.sqrt(periods)))


#Generate d1 column
for q in range(0,13):
    d1.append((np.log(S[q]/K)+(r+0.5*sigma**2)*tau[q]) / (sigma*np.sqrt(tau[q])))
    
    delta.append(norm.cdf(d1[q]))
    ds.append(delta[q]*S[q])


#Generate cash and portoflio columns   
cash=[C0-ds[0]]

for c in range(0,12):
    cash.append(cash[c]-(delta[c+1]-delta[c])*S[c+1])
  

#generate BS Call price evolution
for u in range(0,len(cash)):
    portfolio.append(ds[u]+cash[u])
    bs.append(get_call(tau[u], 0, 0, S[u],106, 0.17))

for p in range(0,13):
    pnl.append(portfolio[p]-bs[p])
    
final_pnl=ds[-1]+cash[-1]-(np.maximum(S[-1]-K,0))
    


s = {'S':S,'d1':d1,'delta':delta,'ds':ds,'cash':cash,'port':portfolio,'bs':bs,'pnl':pnl}
pd.DataFrame(s)




        S      d1  delta     ds    cash  port    bs   pnl
0  100.00   -0.26   0.40  39.83  -35.45  4.38  4.38  0.00
1  108.23    0.21   0.58  63.08  -55.42  7.66  8.13 -0.47
2  117.14    0.72   0.77  89.61  -76.76 12.85 13.84 -0.98
3  132.50    1.59   0.94 125.08 -100.47 24.60 26.99 -2.39
4  134.21    1.77   0.96 129.00 -102.79 26.22 28.51 -2.30
5  129.46    1.61   0.95 122.50 -100.84 21.66 23.86 -2.21
6  114.57    0.71   0.76  87.09  -79.53  7.56 10.64 -3.07
7  115.13    0.81   0.79  90.92  -82.93  7.99 10.73 -2.74
8  118.49    1.19   0.88 104.60  -93.96 10.64 13.18 -2.54
9  109.31    0.40   0.66  71.82  -69.28  2.54  5.54 -3.00
10 111.63    0.77   0.78  87.13  -83.06  4.07  6.65 -2.59
11 106.77    0.17   0.57  60.75  -60.49  0.27  2.45 -2.18
12 102.98 -169.75   0.00   0.00   -1.89 -1.89  0.00 -1.89
